In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import re
from matplotlib import pyplot as plt

In [2]:
dataset = pd.read_csv('news_summary.csv')#,encoding="ISO-8859-1")
dataset.head()

,author,date,headlines,read_more,text,ctext
0,Chhavi Tyagi,"03 Aug 2017,Thursday",Daman & Diu revokes mandatory Rakshabandhan in...,http://www.hindustantimes.com/india-news/raksh...,The Administration of Union Territory Daman an...,The Daman and Diu administration on Wednesday ...
1,Daisy Mowke,"03 Aug 2017,Thursday",Malaika slams user who trolled her for 'divorc...,http://www.hindustantimes.com/bollywood/malaik...,Malaika Arora slammed an Instagram user who tr...,"From her special numbers to TV?appearances, Bo..."
2,Arshiya Chopra,"03 Aug 2017,Thursday",Virgin' now corrected to 'Unmarried' in IGIMS'...,http://www.hindustantimes.com/patna/bihar-igim...,The Indira Gandhi Institute of Medical Science...,The Indira Gandhi Institute of Medical Science...
3,Sumedha Sehra,"03 Aug 2017,Thursday",Aaj aapne pakad liya: LeT man Dujana before be...,http://indiatoday.intoday.in/story/abu-dujana-...,Lashkar-e-Taiba's Kashmir commander Abu Dujana...,Lashkar-e-Taiba's Kashmir commander Abu Dujana...
4,Aarushi Maheshwari,"03 Aug 2017,Thursday",Hotel staff to get training to spot signs of s...,http://indiatoday.intoday.in/story/sex-traffic...,Hotels in Maharashtra will train their staff t...,Hotels in Mumbai and other Indian cities are t...


In [3]:
dataset = dataset.dropna(how='any',axis=0) 
x = dataset['ctext']
y = dataset['text']
del dataset

In [4]:
def preprocess_sentence(w):
    w = w.lower().strip()

    # creating a space between a word and the punctuation following it
    # eg: "he is a boy." => "he is a boy ."
    # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
    w = re.sub(r"([?.!,¿])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)

    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
    w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)

    w = w.rstrip().strip()

    # adding a start and an end token to the sentence
    # so that the model know when to start and stop predicting.
    w = '<start> ' + w + ' <end>'
    return w

In [5]:
def tokenize(lang):
    lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
  
    lang_tokenizer.fit_on_texts(lang)

    tensor = lang_tokenizer.texts_to_sequences(lang)

    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')

    return tensor, lang_tokenizer

In [6]:
x = [preprocess_sentence(w) for w in x]
y = [preprocess_sentence(w) for w in y]
x_train, inp_lang = tokenize(x)
y_train, targ_lang = tokenize(y)
print("Input Tensor Shape :" , x_train.shape , "Target Tensor Shape :" , y_train.shape)
print("Preprocessed Text : \n" , x[0])
print("Tokenized Text : \n" , x_train[0])

Input Tensor Shape : (4395, 5791) Target Tensor Shape : (4395, 86)
Preprocessed Text : 
 <start> the daman and diu administration on wednesday withdrew a circular that asked women staff to tie rakhis on male colleagues after the order triggered a backlash from employees and was ripped apart on social media . the union territory ? s administration was forced to retreat within hours of issuing the circular that made it compulsory for its staff to celebrate rakshabandhan at workplace . ? it has been decided to celebrate the festival of rakshabandhan on august . in this connection , all offices departments shall remain open and celebrate the festival collectively at a suitable time wherein all the lady staff shall tie rakhis to their colleagues , ? the order , issued on august by gurpreet singh , deputy secretary personnel , had said . to ensure that no one skipped office , an attendance report was to be sent to the government the next evening . the two notifications ? one mandating the ce